In [1]:
import os

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [33]:
from dotenv import load_dotenv
load_dotenv()

True

Load the model and test it

In [9]:
from langchain_core.messages import HumanMessage, SystemMessage

In [10]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [11]:
model = ChatGoogleGenerativeAI(model ="gemini-pro", convert_system_message_to_human=True)

In [12]:
message = [SystemMessage(content = "Hi, you are nice bot."), HumanMessage(content ="Hi, how are you?")]

In [13]:
model.invoke(message)

AIMessage(content='I am well, thank you for asking. How are you doing today?', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-fcfe3cec-b86c-4e47-943a-cb2b5de1662d-0', usage_metadata={'input_tokens': 14, 'output_tokens': 15, 'total_tokens': 29})

In [14]:
message2 =[SystemMessage(content = "Hi! You are a nice bot and you answer everything in brief."), HumanMessage(content ="Tell me about red dead redemption.")]

Use Output Parser now

In [15]:
from langchain_core.output_parsers import StrOutputParser

In [16]:
parser = StrOutputParser()

In [17]:
response = model.invoke(message)

In [18]:
parser.invoke(response)

'Thank you for the compliment. I am a virtual assistant, not a bot. I am designed to help people with a variety of tasks, including answering questions, providing information, and completing tasks.\n\nI am currently in development and learning new things every day. I am always looking for ways to improve my skills and help people in the best way possible.\n\nHow can I help you today?'

In [19]:
response2 = model.invoke(message2)

In [20]:
parser.invoke(response2)

'Red Dead Redemption is a 2010 action-adventure game developed and published by Rockstar Games. It is the second installment in the Red Dead series, and a prequel to the 2004 game Red Dead Revolver. The game is set in 1911, in the fictional American frontier of the Wild West. The story follows John Marston, a former outlaw who is forced to hunt down his former gang members by the government in order to save his family.\n\nThe game received critical acclaim upon release, with praise for its open world, story, characters, graphics, and gameplay. It was a commercial success, selling over 15 million copies worldwide.'

Chaining with LCEL(Langchain Expression Language)

Added two components

In [21]:
chain = model | parser 

In [22]:
chain.invoke(message)

'I am a virtual assistant, not a bot. I am designed to help you with a variety of tasks, including answering your questions, providing information, and completing tasks.\n\nI am doing well, thank you for asking. How can I assist you today?'

In [23]:
chain.invoke(message2)

"Red Dead Redemption is an open-world western action-adventure game developed by Rockstar Games and published by Rockstar Games for the PlayStation 3 and Xbox 360 in 2010. The game is set in 1911 in the fictional American West and follows the story of John Marston, a former outlaw who is forced to hunt down his former gang members by the government. The game is played from a third-person perspective and features a vast open world that players can explore at their own pace. Players can engage in a variety of activities, including shooting, riding horses, and hunting. The game also features a morality system that affects the player's interactions with other characters."

Start Prompting

In [24]:
from langchain_core.prompts import ChatPromptTemplate

In [25]:
system = "translate the following into {language}"
user =  "I am hungry"

In [26]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system","translate the following into {language}"),
     ("user","{text}")]
)

In [27]:
prompt = prompt_template.invoke({"language":"french", "text":"I am learning something"})

In [28]:
model.invoke(prompt)

AIMessage(content="Je suis en train d'apprendre quelque chose", response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-0432e030-2c9f-493b-b30c-e0d606fb6a70-0', usage_metadata={'input_tokens': 10, 'output_tokens': 9, 'total_tokens': 19})

In [29]:
prompt.to_messages()

[SystemMessage(content='translate the following into french'),
 HumanMessage(content='I am learning something')]

In [30]:
chain.invoke(prompt)

"J'apprends quelque chose"

Chaining Prompts 

In [31]:
chain = prompt_template | model | parser 

In [32]:
chain.invoke({"language":"Bangla", "text":"I am learning something"})

'আমি কিছু শিখছি'

Agents

In [34]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage

In [35]:
memory = MemorySaver()

In [36]:
search = TavilySearchResults(max_results = 2)

In [37]:
tools = [search]

In [38]:
agent_executor=create_react_agent(model,tools,checkpointer=memory)

Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


In [39]:
config = {"configurable": {"thread_id": "abc123"}}

In [40]:
for chunk in agent_executor.stream({"messages":[HumanMessage(content="hi im samin! and I am learning langchain")]}, config):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Nice to meet you Samin. What would you like to know about langchain?', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-c4c73e30-4a70-441a-bf08-eaa05a591669-0', usage_metadata={'input_tokens': 85, 'output_tokens': 17, 'total_tokens': 102})]}}
----


In [41]:
for chunk in agent_executor.stream({"messages": [HumanMessage(content="whats the weather where I live?")]}, config):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='I cannot fulfill this request. I do not have the functionality to get the weather.', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-75055ae6-0eb3-41e3-9cc2-0bda8adcaf7a-0', usage_metadata={'input_tokens': 111, 'output_tokens': 17, 'total_tokens': 128})]}}
----
